# **Intro to SiS1 Labs**

The goal of the labs of Signal and Systems 1 is to gain knowledge on the course’s topics by solving a specific signal processing problem. By the end of the course, you should be able to imitate, by synthesis, the sound produced by a plucked string instrument using the knowledge acquired in the theory and seminar classes. This will be a cumulative process, in each lab new techniques will be applied in order to improve the synthesis. First let's show what we will achieve by the end of the course. For each lab we need to import some auxiliary functions needed to load the audio files and listen to them:


In [ ]:
from util import load_audio, save_audio, plot_signals
from IPython.display import Audio

**Reference signal**

Now let's load and listen to a reference signal like the one you will use:

In [ ]:
reference, fs = load_audio('audio/reference.wav')
Audio(reference, rate=fs)

**Synthesis**

Now we can load and listen to a synthesised signal like the one you will obtain and that tries to imitate the reference signal. This is an audio signal created from scratch only using the knowledge gained during the course.

In [ ]:
synthesis, fs = load_audio('audio/synthesis.wav')
Audio(synthesis, rate=fs)

---

## **Lab 1**
In this lab you will choose your reference sound to be used for all the labs and start imitating it with a single sinusoid.

## **Exercises**

**1. Choose a reference sound**

1.1. Find a recording of a single plucked string sound. Be creative, do not choose “guitar” by default, nor copy a sound from another group. You should use [Freesound](https://freesound.org/) to find the recordings (Freesound is a huge collaborative database of audio snippets released under Creative Commons licenses created and maintained by researchers of the Music Technology Group of the UPF). Try to find a sound with the best possible quality. The ideal sound format should be: uncompress (such as.wav, .aiff, but no .mp3), mono (one channel) and with a sampling rate of  44100Hz. The sound file should also contain a single note. You can start by downloading a sound from freesound and modify it with Audacity to the right format and content.

https://freesound.org/people/Jagadamba/sounds/253767/

1.2. Create a github repository and upload your edited audio file there:

1) Sing up to github using your UPF gmail account: https://github.com/

2) Create a repository called **sis1_groupX** where X is your group number: https://docs.github.com/en/get-started/quickstart/create-a-repo

3) Upload your edited audio file to the repository


1.3. Now you can clone your repository and access the audio file locally. Change the following code (user name and file name) to clone the repository and load the audio file.

In [ ]:
filepath = "./sis1_group102/audio/audioSisLabs.wav"
ref, fs = load_audio(filepath)


: 

Note that `load_audio` function returns two variables: the audio signal (as a one dimensional array of floating point numbers), and the sampling rate (as a integer number). Now we can plot the audio signal:

In [ ]:
plot_signals(ref, fs)

You can use the zoom in tool to see more details.

You can also use Audio widget to listen the audio signal:

In [ ]:
Audio(ref, rate=fs)



---



**2. Measure the signal period and fundamental frequency**

2.1. Measure the period length, in seconds, and the fundamental frequency, in Hz, of the sound you choose. You can use Audacity to measure the period by zooming into a stable portion of the sound. Then compute its inverse to find the fundamental frequency. Find the closest note (note name) of the measured frequency (use google).

Fundamental frequency: ≈ 374 Hz.
Closest musical note (A4 = 440 Hz): F♯4 / G♭4 (theoretical 369.99 Hz).

2.2. Calculate the period from the frequency value using Python:


In [ ]:
import numpy as np
from util import load_audio, plot_signals  # adjust import path if needed

x, fs = load_audio("./sis1_group102/audio/audioSisLabs.wav")

t0, dur = 0.12, 0.35
i0, i1 = int(t0*fs), int((t0+dur)*fs)
seg = x[i0:i1] - np.mean(x[i0:i1])

n = 1 << int(np.ceil(np.log2(2*len(seg))))
X = np.fft.rfft(seg, n=n)
r = np.fft.irfft(np.abs(X)**2, n=n)[:len(seg)]
r = r / (r[0] + 1e-12)

fmin, fmax = 60.0, 1500.0
lag_min, lag_max = int(fs/fmax), min(len(r)-1, int(fs/fmin))
lag = np.argmax(r[lag_min:lag_max+1]) + lag_min

f0 = fs / lag
T = 1.0 / f0
print(f"f0 = {f0:.2f} Hz, T = {T*1e3:.2f} ms")

plot_signals([x], fs=fs, t_start=i0/fs, t_end=i0/fs + 3*T, labels=["reference"])


2.3. Plot three periods of the signal using the `plot_signals` function and selecting `t_start` and `t_end` arguments accordingly. Try to get the first periods of the audio signal but avoiding the attack section.

In [ ]:
Tplot = T
t_start = (i0/fs) + 0.02          
t_end   = t_start + 3*Tplot

if plot_signals:
    plot_signals([ref], fs=fs, t_start=t_start, t_end=t_end, labels=["reference"])
else:
    n = len(ref)
    t = np.arange(n)/fs
    i_a = max(0, int(t_start*fs))
    i_b = min(n, int(t_end*fs))
    plt.figure()
    plt.plot(t[i_a:i_b], ref[i_a:i_b])
    plt.xlabel("Time (s)"); plt.ylabel("Amplitude")
    plt.title("Three periods of the reference (attack avoided)")
    plt.show()


2.4. Measure the period of the signal manually by identifying the begining and ending of a periodic cycle. Note that when you hover the mouse over one of the points in the plot, you can see the time and amplitude values. For instance, you can check the cross by zero.

After zooming into a steady segment and checking the distance between consecutive upward zero crossings, the measured period is approximately 2.642 milliseconds. This corresponds to a measured frequency of approximately 378.5 hertz.

2.5. Do the measured (2.4) and theoretical (2.2) periods coincide? If not, explain why.

The measured and theoretical values are very close but not identical. From question 2.2 the theoretical period is approximately 2.676 milliseconds and the theoretical frequency is approximately 373.7 hertz. The absolute difference between periods is about 0.034 milliseconds which is about 1.27 percent of the theoretical period. The absolute difference between frequencies is about 4.81 hertz which is about 1.29 percent of the theoretical frequency. The small mismatch is explained by the window you choose for measurement, the sound not being a perfectly pure sinusoid because it has harmonics and a decaying envelope, finite sampling effects, and small manual readout inaccuracies.



---



**3. Generating a sinusoid**

3.1. Create a sinusoid of the same duration than the reference signal and same frequency than the fundamental frequency. Plot it along with the audio signal of the reference recording. Plot the same time segment that 2.3. Note that the sampling rate should be the same for both signals. Try to find the values of Amplitude and initial Phase that make the two signal segments plotted to match as close as possible (do not attempt to imitate the whole signal). Matching the phase is a bit tricky.



In [ ]:
N = len(ref); t_all = np.arange(N)/fs; w = 2*np.pi*f0
ta, tb = t_start, t_end
m = (t_all >= ta) & (t_all <= tb)
tm = t_all[m]; xm = ref[m]
C = np.column_stack([np.cos(w*tm), np.sin(w*tm)])
a, b = np.linalg.lstsq(C, xm, rcond=None)[0]
A_fit = np.hypot(a, b); phi_fit = np.arctan2(-b, a)
synth = A_fit * np.cos(w*t_all + phi_fit)
if plot_signals:
    plot_signals([ref, synth], fs=fs, t_start=t_start, t_end=t_end, labels=["reference", "synth"])
else:
    ia, ib = int(t_start*fs), int(t_end*fs)
    plt.figure(); plt.plot(t_all[ia:ib], ref[ia:ib], label="reference")
    plt.plot(t_all[ia:ib], synth[ia:ib], label="synth", alpha=0.8)
    plt.legend(); plt.xlabel("Time (s)"); plt.ylabel("Amplitude")
    plt.title(f"A={A_fit:.3f}, phi={phi_fit:.3f} rad, f0={f0:.2f} Hz"); plt.show()
print(f"A={A_fit:.4f}, phi={phi_fit:.4f} rad, f0={f0:.2f} Hz")


3.2. What are the main differences between the two signals.

[Double click to enter your answer]


3.3. Listen to the synthesized signal usign the Audio widget. Sounds natural?

In [ ]:
Audio(synth, rate=fs)


3.4. Save the synthesized signal using the 'save_audio` function:

In [ ]:
out_path = "./sis1_group102/audio/synthesis.wav"
s_out = np.clip(synth, -1.0, 1.0)
if save_audio:
    save_audio(s_out, fs, out_path)
else:
    import soundfile as sf
    sf.write(out_path, s_out, fs)
print("Saved:", out_path)
